In [1]:
%matplotlib notebook
import os
import os.path as path
import numpy as np
from scipy.optimize import curve_fit
from faker import Factory
from scipy import interpolate
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
import h5py

from pyccapt.calibration.calibration_tools import tools, data_tools, variables, tof_calibration, data_loadcrop
from pyccapt.calibration.calibration_tools import widgets as wd


In [2]:
number_sample = 300
save_fig=False
plot_fig=True 

In [3]:
# flightPathLength = 110 # mm 
# # The initial value for t_0
# t0 = 51.74 # ns

# dataset name
# AL_data_b
# OLO_AL_6_data
# OLO_W_6_data
# OLO_Ni_8_data
# X6Cr17_2V30Min_5_data
# data_1029_Jun-27-2022_15-53_Pd
# data_115_Jul-27-2022_17-44_Powersweep3

In [4]:
tdc, pulse_mode, dataset, flightPathLength_d, t0_d = wd.dataset_tdc_selection()
display(tdc, pulse_mode, dataset, flightPathLength_d, t0_d)

Dropdown(description='TDC model:', options=('surface_concept', 'roentdec'), value='surface_concept')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

Text(value='OLO_AL_6_data', description='Dataset:', placeholder='Paste ticket description here!')

Text(value='110', description='Flight path length:', placeholder='Flight path length')

Text(value='51.74', description='t0:', placeholder='T_0 of the instrument')

$$\textbf{You can specify which dataset to use in below block}$$

In [5]:
tdc_model = tdc.value
pulse_mode_ini = pulse_mode.value

dataset_name = dataset.value

flightPathLength = float(flightPathLength_d.value) # mm 
# The initial value for t_0
t0 = float(t0_d.value) # ns

p = path.abspath(path.join("", "../../../.."))

variables.init()

path_main = os.path.join(p, 'tests//data')
filename_main = path_main + '//' + dataset_name + '.h5'

# variables.path = os.path.join(p, 'tests//results//mc_vol_bowl_calibratin')
variables.path = os.path.join(p, 'tests//results//tof_calibration')
variables.result_path = os.path.join(p, 'tests/results/ion_type_selection/' + dataset_name)
if not os.path.isdir(variables.result_path):
        os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
        
filename = variables.path + '//' + dataset_name + '//' + dataset_name + '.h5'



head, tail = os.path.split(filename)
figname = os.path.splitext(tail)[0]

data = data_tools.read_hdf5_through_pandas(filename)


In [6]:
data

,high_voltage (V),pulse (V),start_counter,t (ns),x (mm),y (mm),pulse_pi,ion_pp,mc (Da)
0,5267.498047,5267.498047,54217.0,642.333996,19.515918,-28.812245,0.0,1.0,27.045662
1,5267.498047,5267.498047,55251.0,622.754406,3.279184,4.170612,1034.0,1.0,26.936527
2,5267.498047,5267.498047,55287.0,638.987292,21.203265,-21.457959,36.0,1.0,27.215287
3,5267.498047,5267.498047,55309.0,623.385342,5.953469,6.813061,22.0,1.0,26.903841
4,5267.498047,5267.498047,56063.0,628.576848,17.955918,-11.620408,754.0,1.0,27.002715
...,...,...,...,...,...,...,...,...,...
1891028,5710.761719,5710.761719,39864.0,603.901764,-11.811429,-3.533878,40.0,1.0,26.901347
1891029,5710.761719,5710.761719,40013.0,604.951038,-12.989388,-2.451429,149.0,1.0,26.926814
1891030,5710.761719,5710.761719,40267.0,619.167672,29.894694,10.442449,254.0,1.0,27.078288
1891031,5710.761719,5710.761719,40371.0,605.712276,-12.798367,-0.986939,104.0,1.0,26.990346


In [7]:
mc = data['mc (Da)'].to_numpy()

In [8]:
peaks_sides_p = np.zeros(0)
def mc_his(bin_size, prominence, distance, percent, plot, figname, mc, mc_ideal):

    bin_size = float(bin_size)
    prominence = int(prominence)
    distance = int(distance)
    percent = int(percent)
    if mc_ideal == True:
        mc_ideal = np.array(variables.listMaterial)
        print('List of mc are:', mc_ideal)
    else:
        mc_ideal = np.zeros(0)
    max_hist, left_right_peaks, peaks_sides, max_paek_edges, index_max = tools.massSpecPlot(mc[mc < 100], bin_size, mc_ideal, distance=distance, percent=percent, prominence=prominence, plot=plot, label='tof', fig_name=figname)
    mrp = (max_hist / (left_right_peaks[1] - left_right_peaks[0]))
    print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
    for i in range(len(peaks_sides)):
        print('Peaks ', i, 'is: {:.2f}'.format(peaks_sides[i,0]), 'peak window sides are: {:.2f} - {:.2f}'.format(peaks_sides[i,2], peaks_sides[i,3]))
    global peaks_sides_p
    peaks_sides_p = peaks_sides

In [9]:
peaks_sides = interact_manual(mc_his, bin_size=widgets.Textarea(value="0.1")
               ,prominence=widgets.Textarea(value="60"), distance=widgets.Textarea(value="100")
               ,percent=widgets.Textarea(value="20"), plot=fixed(True), mc=fixed(mc),  figname=fixed(figname)
                , mc_ideal=[('False',False),('True',True)]);



interactive(children=(Textarea(value='0.1', description='bin_size'), Textarea(value='60', description='promine…

In [10]:
isotopeTableFile = '../../../files/isotopeTable.h5'
dataframe = data_tools.read_hdf5_through_pandas(isotopeTableFile)
elementsList = dataframe['element']
elementIsotopeList = dataframe['isotope']
elementMassList =  dataframe['weight']
abundanceList = dataframe['abundance']

elements = list(zip(elementsList, elementIsotopeList, elementMassList, abundanceList))
dropdownList = []
for element in elements:
    tupleElement = ("{} ({}) ({:.2f})".format(element[0],element[1],element[3]), "{}{}".format(element[0],element[1]))
    dropdownList.append(tupleElement)

chargeList = [(1,1,),(2,2,),(3,3,),(4,4,)]
dropdown = wd.dropdownWidget(dropdownList, "Elements")
dropdown.observe(wd.on_change_ions_selection)


chargeDropdown = wd.dropdownWidget(chargeList, "Charge")
chargeDropdown.observe(wd.on_change_charge_ions_selection)

wd.compute_element_isotope_values_according_to_selected_charge(mode='ions_selection')

buttonAdd = wd.buttonWidget("ADD")
buttonDelete = wd.buttonWidget("DELETE")
buttonReset = wd.buttonWidget("RESET")

display(dropdown)
display(chargeDropdown)
display(buttonAdd)
display(buttonDelete)
display(buttonReset)
listMaterial = buttonAdd.on_click(wd.onClickAdd)
buttonDelete.on_click(wd.onClickDelete)
buttonReset.on_click(wd.onClickReset)

Dropdown(description='Elements', options=(('H (1) (99.98)', 'H1'), ('H (2) (0.01)', 'H2'), ('He (3) (0.00)', '…

Dropdown(description='Charge', options=((1, 1), (2, 2), (3, 3), (4, 4)), value=1)

Button(description='ADD', icon='check', style=ButtonStyle(), tooltip='ADD')

Button(description='DELETE', icon='check', style=ButtonStyle(), tooltip='DELETE')

Button(description='RESET', icon='check', style=ButtonStyle(), tooltip='RESET')

In [11]:
fake = Factory.create()
color = np.zeros(len(mc), dtype='object' )
element = np.zeros(len(mc), dtype='object')

for index, elemen in enumerate(variables.listMaterial):
    mask = np.logical_and((peaks_sides_p[index,2] < mc), (mc < peaks_sides_p[index,3])).squeeze()
    print(mask[mask==True].shape)

    index_true = np.where(mask==True)
    index_get_to_false = np.random.choice(index_true[0], size=int(index_true[0].shape[0] - index_true[0].shape[0]), replace=False)
    mask[index_get_to_false] = False
    print('ploted ions', mask[mask==True].shape)
    
    color[mask] = fake.hex_color()
    element[mask] = elemen



(965,)
ploted ions (965,)
(140033,)
ploted ions (140033,)
(1492747,)
ploted ions (1492747,)


In [12]:
data['element'] = element
data['color'] = color

In [13]:
data

,high_voltage (V),pulse (V),start_counter,t (ns),x (mm),y (mm),pulse_pi,ion_pp,mc (Da),element,color
0,5267.498047,5267.498047,54217.0,642.333996,19.515918,-28.812245,0.0,1.0,27.045662,Al27(1+),#43bd1b
1,5267.498047,5267.498047,55251.0,622.754406,3.279184,4.170612,1034.0,1.0,26.936527,Al27(1+),#43bd1b
2,5267.498047,5267.498047,55287.0,638.987292,21.203265,-21.457959,36.0,1.0,27.215287,Al27(1+),#43bd1b
3,5267.498047,5267.498047,55309.0,623.385342,5.953469,6.813061,22.0,1.0,26.903841,Al27(1+),#43bd1b
4,5267.498047,5267.498047,56063.0,628.576848,17.955918,-11.620408,754.0,1.0,27.002715,Al27(1+),#43bd1b
...,...,...,...,...,...,...,...,...,...,...,...
1891028,5710.761719,5710.761719,39864.0,603.901764,-11.811429,-3.533878,40.0,1.0,26.901347,Al27(1+),#43bd1b
1891029,5710.761719,5710.761719,40013.0,604.951038,-12.989388,-2.451429,149.0,1.0,26.926814,Al27(1+),#43bd1b
1891030,5710.761719,5710.761719,40267.0,619.167672,29.894694,10.442449,254.0,1.0,27.078288,Al27(1+),#43bd1b
1891031,5710.761719,5710.761719,40371.0,605.712276,-12.798367,-0.986939,104.0,1.0,26.990346,Al27(1+),#43bd1b


In [14]:
# save the new data
name_save_file = variables.result_path + '//' + dataset_name + '.h5'
data_tools.store_df_to_hdf(name_save_file, data, 'df')

c:\users\mehrp\pycharmprojects\pyccapt\pyccapt\calibration\calibration_tools\data_tools.py:91: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['element', 'color'], dtype='object')]

  dataframe.to_hdf(filename, key, mode='w')


In [15]:
data.dtypes

high_voltage (V)    float64
pulse (V)           float64
start_counter       float64
t (ns)              float64
x (mm)              float64
y (mm)              float64
pulse_pi            float64
ion_pp              float64
mc (Da)             float64
element              object
color                object
dtype: object